In [1]:
import torch
from torch.optim import Adam
from torch import nn
from torchvision import transforms
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
import numpy as np
import time
from tqdm import tqdm

from ExpressionDataset import ExpressionDataset

In [2]:
device = 'cuda'
print(torch.cuda.get_device_name(0))

image_size = 96
mean = [0.5375, 0.4315, 0.3818]
std = [0.2928, 0.2656, 0.2614]

epochs = 500

NVIDIA GeForce RTX 3090 Ti


In [3]:
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

dataset = ExpressionDataset('./data/', transform=transform)
dataset = [dataset[i] for i in torch.randperm(len(dataset))]
train_len = int(len(dataset) * 0.8)
val_len = len(dataset) - train_len
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_len, val_len])
train_dl = DataLoader(train_dataset, batch_size=16, num_workers=8)
val_dl = DataLoader(val_dataset, batch_size=128, num_workers=8)

In [4]:
# Using a pretrained (open source) model for the sake of time
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=2)
model.set_swish(memory_efficient=False)
model = model.to('cuda')

Loaded pretrained weights for efficientnet-b0


In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min')

In [6]:
def train(model, loader, criterion, optimizer):
    train_loss = 0.0
    train_correct = 0

    for images, labels in tqdm(train_dl):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)

        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_correct += torch.sum(predictions == labels.data)

    return train_loss / len(train_dl)

In [7]:
from sklearn.metrics import f1_score, roc_auc_score, recall_score

def evaluate(model, loader):
    val_loss = 0.0
    val_correct = 0

    model.eval()

    label_list = []
    pred_list = []

    with torch.no_grad():
        for images, labels in tqdm(loader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predictions = torch.max(outputs, 1)

            loss = criterion(outputs, labels)

            val_loss += loss.item()
            val_correct += torch.sum(predictions == labels.data)
            label_list = np.concatenate([label_list, labels.cpu()])
            pred_list = np.concatenate([pred_list, predictions.cpu()])

    val_accuracy = (100.0 * val_correct / len(loader.dataset)).cpu()
    val_f1 = f1_score(label_list, pred_list)
    val_roc_auc = roc_auc_score(label_list, pred_list)
    val_recall = recall_score(label_list, pred_list)

    return val_accuracy, val_recall, val_f1, val_roc_auc


In [ ]:
train_hist = []
val_hist = []

for epoch in range(1, epochs + 1):
    s = time.time()
    loss = train(model, train_dl, criterion, optimizer)
    train_acc, train_recall, train_f1, train_roc_auc = evaluate(model, train_dl)
    val_acc, val_recall, val_f1, val_roc_auc = evaluate(model, val_dl)
    scheduler.step(loss)
    e = time.time()

    train_hist.append((train_acc, train_recall, train_f1, train_roc_auc))
    val_hist.append([val_recall])

    if sum([val_recall]) >= np.asarray(val_hist).sum(axis=1).max():
        print("Saving Best Performing Model...")
        torch.save(model.state_dict(), './models/BestModel.pt')

    print(f'Epoch: {epoch:03d}, Loss: {loss:.05f}\n'
          f'Train Acc: {train_acc}, Train Recall: {train_recall}, Train F1: {train_f1}, Train ROC-AUC: {train_roc_auc}\n'
          f'Val Acc: {val_acc}, Val Recall: {val_recall}, Val F1: {val_f1}, Val ROC-AUC: {val_roc_auc}\n'
          f'Time: {e - s:.05f} seconds')

100%|██████████| 46/46 [00:23<00:00,  2.00it/s]


Saving Best Performing Model...
Epoch: 001, Loss: 0.20189
Train Acc: 97.7962417602539, Train Recall: 0.9310770750988142, Train F1: 0.9363975155279504, Train ROC-AUC: 0.9594660851126074
Val Acc: 96.07505798339844, Val Recall: 0.8664658634538153, Val F1: 0.8833162743091095, Val ROC-AUC: 0.9233638272182851
Time: 168.30545 seconds


100%|██████████| 46/46 [00:22<00:00,  2.02it/s]


Epoch: 002, Loss: 0.08632
Train Acc: 98.40744018554688, Train Recall: 0.9214426877470355, Train F1: 0.9527458492975733, Train ROC-AUC: 0.9593661184369788
Val Acc: 96.72920989990234, Val Recall: 0.857429718875502, Val F1: 0.8998946259220232, Val ROC-AUC: 0.9237283645281312
Time: 165.72241 seconds


100%|██████████| 46/46 [00:22<00:00,  2.03it/s]


...

100%|██████████| 46/46 [00:22<00:00,  2.03it/s]


Saving Best Performing Model...
Epoch: 005, Loss: 0.02128
Train Acc: 99.1262435913086, Train Recall: 0.9930830039525692, Train F1: 0.9753730437947349, Train ROC-AUC: 0.9919806471417786
Val Acc: 96.55706787109375, Val Recall: 0.9377510040160643, Val F1: 0.9032882011605416, Val ROC-AUC: 0.9545393291428753
Time: 150.85913 seconds


100%|██████████| 46/46 [00:21<00:00,  2.15it/s]


Epoch: 006, Loss: 0.01617
Train Acc: 99.5523681640625, Train Recall: 0.9757905138339921, Train F1: 0.9870064967516242, Train ROC-AUC: 0.9877388847512416
Val Acc: 96.86692810058594, Val Recall: 0.8594377510040161, Val F1: 0.9039070749736009, Val ROC-AUC: 0.9253556924560906
Time: 143.07582 seconds


  0%|          | 0/1453 [00:00<?, ?it/s]

In [ ]:
model.eval()
example_input = torch.randn(1, 3, 96, 96).cuda()

# Export the model
torch.onnx.export(model,                     # model being run
                  example_input,             # model input (or a tuple for multiple inputs)
                  "./models/model.onnx",     # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=11,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names=['input'],     # the model's input names
                  output_names=['output'],   # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})